In [238]:
from pulp import *
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import geopandas
import folium
from shapely.geometry import Point

## **Establecer Parámetros**

In [239]:
n = 7 # Cantidad de clientes
m = 44 # Cantidad de proveedores

# Lista de clientes
posClientes = [(25.653809, -100.376382), (25.688748, -100.316980), (25.750746, -100.295240),
               (25.670246, -100.241593), (25.759089, -100.197792), (25.682101, -100.439690),
               (25.797502, -100.580751)]
clientesX = [posClientes[i][0] for i in range(n)]
clientesY = [posClientes[i][1] for i in range(n)]
# 1: San Pedro
# 2: Monterrey
# 3: San Nicolás
# 4: Guadalupe
# 5: Apodaca
# 6: Santa Catarina
# 7: García



# Lista de proveedores
posProvee = [(25.671379, -100.346960), (25.641851, -100.321602), (25.650777, -100.332919), (25.666469, -100.317415), (25.66966055033981, -100.37471414317174), (25.650087138173415, -100.33422648632937),
             (25.676220368251865, -100.2917754668386), (25.64285775341181, -100.32284279187544), (25.64879292411893, -100.33418813420428), (25.648204268328467, -100.35337517468467),
             (25.65441674280337, -100.33737274581655), (25.671380200426054, -100.34643492695288), (25.653379906941286, -100.34908839967825), (25.674966646628647, -100.36494029954568),
             (25.67364637049978, -100.35584205934197), (25.67022962250775, -100.37423926195622), (25.658879223212043, -100.44180821488823), (25.645448, -100.322931),
             (25.648566067800402, -100.33329291886224), (25.66017597538397, -100.34185663235534), (25.61806395730135, -100.30509766119188), (25.648564245370583, -100.33305989085628),
             (25.618838819254623, -100.30420580721815), (25.644532929645766, -100.32107677653333), (25.63361149687988, -100.28286590324389), (25.674485686715904, -100.31143214399988),
             (25.652276919873763, -100.35223233050694), (25.652616704621327, -100.35251970352004), (25.65008503646141, -100.35707531967138), (25.67151980411705, -100.37694456119061),
             (25.647409999191193, -100.35379864610911), (25.649499084067973, -100.35182454037394), (25.66955145719424, -100.35337438817758), (25.67442766923161, -100.31156089002607),
             (25.67046166591082, -100.32548188817758), (25.67039948592011, -100.37925227283546), (25.664480326187903, -100.35329721886183), (25.669768549194448, -100.34160778022999),
             (25.64873417116146, -100.32523740352009), (25.648618111106053, -100.32519448817803), (25.654917673817057, -100.4494457728359), (25.65251621858032, -100.3329652630397),
             (25.652073104368668, -100.33312373617782), (25.67315776834992, -100.29875901886163)]
proveeX = [posProvee[i][0] for i in range(m)]
proveeY = [posProvee[i][1] for i in range(m)]


# Diccionario con las demandas
demandas = [100, 80, 70, 40, 40, 40, 10]
demandas = [i*10 for i in demandas]
# Diccionario con las ofertas
ofertas = [110]*m
# Diccionario de precios que tiene cada facility
precios = [100000]*m

## **Imprimir el Mapa Inicial**

In [240]:
m_proveedores = pd.DataFrame()

m_proveedores['Nombre'] = [f'Estación {i}' for i in range(1, m+1)]
m_proveedores['Latitud'] = [i[0] for i in posProvee]
m_proveedores['Longitud'] = [i[1] for i in posProvee]

#Expresamos nuestro dataframe en el formato adecuado
m_geometry = geopandas.points_from_xy(m_proveedores.Longitud, m_proveedores.Latitud)
m_proveedores = geopandas.GeoDataFrame(m_proveedores[['Nombre', 'Latitud', 'Longitud']], geometry=m_geometry)

In [241]:
m_clientes = pd.DataFrame()

m_clientes['Nombre'] = [f'Cliente {i}' for i in range(1, n+1)]
m_clientes['Latitud'] = [i[0] for i in posClientes]
m_clientes['Longitud'] = [i[1] for i in posClientes]

#Expresamos nuestro dataframe en el formato adecuado
c_geometry = geopandas.points_from_xy(m_clientes.Longitud, m_clientes.Latitud)
m_clientes = geopandas.GeoDataFrame(m_clientes[['Nombre', 'Latitud', 'Longitud']], geometry=c_geometry)

## **Preparar las Estructuras de Datos**

In [242]:
max_dig = int(math.ceil(math.log10(m+1)))
# Listas de clientes y proveedores
Customer = [i for i in range(1, n+1)]
Facility = [f'Fac_{i:0>{max_dig}}' for i in range(1, m+1)]

# Diccionario de demandas
Demand = {}
for i in range(n):
    Demand[Customer[i]] = demandas[i]

# Diccionarios de ofertas
Max_Supply = {}
for i in range(m):
    Max_Supply[Facility[i]] = ofertas[i]

# Diccionarios de precio de habilitar proveedores
fixed_cost = {}
for i in range(m):
    fixed_cost[Facility[i]] = precios[i]

In [243]:
# Generar el grafo de distancias
def distanciaEuclideana(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

transportation_cost = {}
for i in range(m): # Proveedores
    aux = {}
    for j in range(n): # Clientes
        aux[Customer[j]] = distanciaEuclideana(posProvee[i], posClientes[j])
    transportation_cost[Facility[i]] = aux

## **Modelo Matemático**

### **Planteamiento**

In [244]:
# Setting the Problem
prob = LpProblem("Optiminizacion_de_Estaciones_de_Recarga", LpMinimize)

In [245]:
# Defining our Desicion Variables
use_facility = LpVariable.dicts("Use Facility", Facility, 0, 1, LpBinary)
ser_customer = LpVariable.dicts("Service", [(i,j) for i in Customer for j in Facility], 0)

In [246]:
# Setting the Objective Function
prob += lpSum(fixed_cost[j]*use_facility[j] for j in Facility) + lpSum(transportation_cost[j][i]*ser_customer[(i,j)] for j in Facility for i in Customer)

In [247]:
# Costraints
for i in Customer:
    prob += lpSum(ser_customer[(i,j)] for j in Facility) == Demand[i]

for j in Facility:
    prob += lpSum(ser_customer[(i,j)] for i in Customer) <= Max_Supply[j]*use_facility[j]

for i in Customer:
    for j in Facility:
        prob += ser_customer[(i,j)] <= Demand[i]*use_facility[j]

### **Resolver**

In [248]:
prob.solve()
print("Status de la solución:", LpStatus[prob.status])

# Print the solution of Binary Decision Variables
Tolerance = 0.000001
for j in Facility:
    if use_facility[j].varValue > Tolerance:
        print("Se establece el proveedor:", j)

print("---------------- Variables de Decisión ----------------")

# Print the solution of Continuous Decision Variables
solution = []
for v in prob.variables():
    print(v.name, "=", v.varValue)
    solution.append(v.varValue)

print("---------------- Costo Total ----------------")

# Print Optimal
print("Costo total:", value(prob.objective))

## **Presentar Resultados**

In [236]:
selectedProvee = solution[-m:]
for i in range(m):
    if selectedProvee[i] == 1: selectedProvee[i] = 'green'
    else: selectedProvee[i] = 'red'

In [237]:
map = folium.Map(location = [m_proveedores['Latitud'].mean(), m_proveedores['Longitud'].mean()], zoom_start = 13)

p_geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in m_proveedores.geometry]
c_geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in m_clientes.geometry]

i = 0
for coordinates in p_geo_df_list:
    #now place the markers with the popup labels and data
    map.add_child(folium.Marker(location = coordinates,
                                icon = folium.Icon(color = selectedProvee[i], icon = 'bolt', prefix='fa'),
                                tooltip = m_proveedores['Nombre'][i]))
    i = i + 1

i = 0
for coordinates in c_geo_df_list:
    map.add_child(folium.Circle(location = coordinates,
                                radius = demandas[i]*5,
                                fill = True,
                                fill_opacity = 0.30,
                                opacity = 0.30,
                                color = "orange",
                                fill_color = "orange",
                                tooltip = m_clientes['Nombre'][i]))
    i += 1

map